#Installation

The installation involves downloading the LatentAudio github repository and installing its dependencies. Important: The dependency tensorflow has to be installed as version 2.9. This version appears to be available only for x86 systems, not for arm. If you do not manage to install this tensorflow version on your local machine, try running it in Google Colab. After the installation it might be necessary to restart the runtime.

In [ ]:
!git clone https://github.com/TimHenry1995/LatentAudio.git

In [ ]:
%cd LatentAudio

In [ ]:
%pip install .

After installation, restart the kernel of the notebook. Then continue from here.

In [ ]:
%cd LatentAudio

/content/LatentAudio


#Processing

Processing is performed mostly by configuring and running existing scripts as shown below. There are two kinds of analysis that need to be prepared, namely latent space exploration and manipulation. 
Latent space exploration involves 
- projection with principal component analysis (PCA),
- t-distributed stochastic neighbor embeddings (t-SNE),
- classification of latent representations as materials and actions using k-nearest neighbors (KNN) and 
- disententanglement with a flow model.

Latent space manipulation involves
- invertible projection with PCA. This requires a complete PCA model whose output and input dimensionality is the same which is processing intensive to set up.
- disentanglement with a flow model. This flow model needs to be invertible. The one listed for exploration will work as it is.

The preparation for these two analyses is similar in many ways. They both require the conversion of data from its waveform domain to Yament's latent representation for each layer and the projection to manageable dimensionality, e.g. 64 dimensions using PCA. Since computing a full PCA model is resoruce intensive and a small model with e.g. 64 dimensions will suffice for most layers. The original latent space representations that are of higher dimensionality than the projection are also no longer needed after projection. Hence, they can be deleted. Only the layers whose latent space shall be manipulated need the full PCA model for invertability and need the original latent space representation. 

2. For each layer l of Yamnet, the following steps need to be executed
  - Convert all sounds to their latent representation at layer l
  - Fit standard scalers and PCA to a sample of these representations. Since the dimensionality of the early Yamnet layers is too large, a complete PCA model cannot be created for them. Yet, a PCA model that only creates the first few dimensions is sufficient to explore the latent space in subsequent experiments. Only for the layers whose original dimensionlaity is sufficiently small, it will be possible to create a PCA model with as many output dimensions as input dimensions. Depending on system resources, this migh twork for layers 4 and 5. It should work for layers 6 and onwards for every reasonably equipped machine.  
  - Project the representations to a more manageable dimensionality
  - Optionally delete the higher dimensional representations. The high dimensional representations are only needed for those layers for which downstream Yamnet processing is intended. Since the original experiment only does this for layer 9, it is not necessary to save these large representations for the remaining layers. As a cosnequence, the final disk storage will be minimized.
  
Important: These steps require several hours to be executed and memory as well as disk storage demands can temporarily peak.

In [2]:
from latent_audio.scripts import audio_to_latent_yamnet as aud2lat, create_scalers_and_PCA_model_for_latent_yamnet as lat2pca, latent_yamnet_to_calibration_data_set as lat2cal
import shutil, os

full_dim_layer_indices = [9]
reduced_target_dimensionality = 64

for layer_index in range(14):
    print(f'Layer {layer_index}')
    # Extract data
    aud2lat.run(layer_index=layer_index) # Converts audio to latent yamnet representation of original dimensionality
    lat2pca.run(layer_index=layer_index, target_dimensionality=None if layer_index in full_dim_layer_indices else reduced_target_dimensionality) # Creates standard scalers and PCA for projection to lower dimensional space
    lat2cal.run(layer_index=layer_index, dimensionality=reduced_target_dimensionality) # Performs the projection (this will be needed for all layers)

    # Delete latent representations of original dimensionality to save disk storage
    if layer_index not in full_dim_layer_indices:
        shutil.rmtree(os.path.join("data","latent yamnet","original",f"Layer {layer_index}"))

Layer 13
Running script to convert audio to latent yamnet
	100.0% Completed
	Run Completed
Running script to create scalers and PCA model for latent yamnet
	Loading sample of latent data Completed. Shape == [instance count, dimensionality] == (10000, 6144)
	Fitting Pre-PCA Standard Scaler to sample Completed
	Fitting 64-dimensional PCA to sample Completed
	Fitting Post-PCA Standard Scaler to sample Completed
	Run Completed
Running script to convert latent yamnet to calibration data set
	The top 64 dimensions explain 83.16 % of variance.
	100.0 % Completed
	Run completed
